# Week 13 (Wednesday), AST 8581 / PHYS 8581 / CSCI 8581: Big Data in Astrophysics

### Ville Cantory <canto063@umn.edu>, Michael Coughlin <cough052@umn.edu>

# Where do we stand?

Foundations of Data and Probability -> Statistical frameworks (Frequentist vs Bayesian) -> Estimating underlying distributions -> Analysis of Time series (periodicity) -> Analysis of Time series (variability) -> Analysis of Time series (stochastic processes) -> Gaussian Processes -> Decision Trees / Regression -> Dimensionality Reduction -> Principle Component Analysis -> Clustering -> Density Estimation / Anomaly Detection -> Supervised Learning -> Deep Learning -> Introduction to Databases - SQL -> Introduction to Databases - NoSQL -> Introduction to Multiprocessing -> Introduction to GPUs

# Tensorflow with GPU

This notebook provides an introduction to computing on a [GPU](https://cloud.google.com/gpu) in Colab. In this notebook you will connect to a GPU, and then run some basic TensorFlow operations on both the CPU and a GPU, observing the speedup provided by using the GPU.


## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [4]:
@vectorize(['float64(float64, float64)'], target='cuda')
def vecCuda_func(a, b):
  for i in range(100000):
    a = math.pow(a*b, 1./2)/math.exp(a*b/1000)
  return a

In [ ]:
%timeit res = vecCuda_func(a, b)

In [ ]:
# Woah!!

# 6. Running your functions on GPU


In [9]:
@cuda.jit
def cudaKernal_func(a, b, result): # cuda.jit does not return result yet
  pos = cuda.grid(1)
  if (pos < a.shape[1]) and (pos < b.shape[0]):
    for i in range(100000):
      result[pos] = math.exp(a[0][pos]*b[pos][0])

In [10]:
result = np.zeros((100,), dtype=np.float64)

In [11]:
threadsperblock = 32
blockspergrid = (100 + 31) // 32 # blockspergrid = (array.size + (threadsperblock - 1)) // threadsperblock

%timeit cudaKernal_func[threadsperblock, blockspergrid](a, b, result)

The slowest run took 6.39 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 55.1 ms per loop


In [12]:
result.shape

(100,)

In [ ]:
#
# Here, we have only used it for 1D arrays. You can use it for any Tensor. For eg:
# For 2D array operations you would have used: x, y = cuda.grid(2)
#

In [13]:
@cuda.jit(device=True)
def cudaDevice_func(a, b):
  for i in range(100000):
    a = math.exp(a*b)
  return a

In [14]:
@cuda.jit
def cudaKernal_func2(a, b, result): # cuda.jit does not return result yet
  pos = cuda.grid(1)
  if (pos < a.shape[1]) and (pos < b.shape[0]):
    result[pos] = cudaDevice_func(a[0][pos], b[pos][0])

In [15]:
%timeit cudaKernal_func2[threadsperblock, blockspergrid](a, b, result)

The slowest run took 6.20 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 58.5 ms per loop


# Initial

### Torch:

In [ ]:
!pip -q install torchvision

### Dask:

In [ ]:
# https://stackoverflow.com/questions/49853303/how-to-install-pydot-graphviz-on-google-colab?rq=1
!pip -q install graphviz 
!apt-get install graphviz -qq
!pip -q install pydot

# Import

In [ ]:
import numpy as np
import pandas as pd
from multiprocessing import Pool, Process
import torch

In [ ]:
import matplotlib.pyplot as plt

# 5. Pycuda (Optional)

In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [ ]:
import numpy
a = numpy.random.randint(1, 9, (10000, 10000))
a = a.astype(numpy.float32) # Most nVidia devices support single precision

In [ ]:
# The main drawback of Pycuda is that you will have to write C code
# to perform any task and pass it to SourceModule...

mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)

# As we saw in Numba post, you have to know which thread we are in to compute index(s) to operate on.
# Here we get that from threadId.

In [ ]:
# Get reference to function and initialize it, passing array and block size (4x4).
func = mod.get_function("doublify")

In [ ]:
a_gpu = cuda.mem_alloc(a.nbytes) # Allocate memory on GPU
cuda.memcpy_htod(a_gpu, a) # Send array to GPU. Now your array is within variable a_gpu
a_gpu = cuda.to_device(a)
func(a_gpu, block=(4, 4, 1))
a_doubled = numpy.empty_like(a) # Copy array from gpu to cpu
cuda.memcpy_dtoh(a_doubled, a_gpu)
a_doubled[0][0:10], a[0][0:10]

In [ ]:
# We can also simply use cuda.InOut which sends array to GPU
# and then retrives back again:
func(cuda.InOut(a), block=(4, 4, 1)) # block takes Block size which can be 3D. Block size gives us number of threads in Block. (No. of threads should be <= 512)
a

In [ ]:
# Whereas a grid is 2D and can have max of 1000's of blocks. As before its dimensions give number of blocks. 

#### Examples:

Image: https://raw.githubusercontent.com/andreajeka/CUDAThreadIndexing/master/images/1dgrid3dblock.png


As shown in image, if we use 1D grid and 3D blocks, that is how you can think of it. Each of the thread sub-blocks contain one thread. Structure like this makes indexing of threads in grid easier.

In [ ]:
#
# 1D grid and 1D block
#

temp = numpy.zeros((4000,))
temp = temp.astype(numpy.float32)
mod = SourceModule("""
  __global__ void cuda_func(float *a)
  {
    int idx = blockDim.x*blockIdx.x + threadIdx.x;  // Go to block + Go to thread
    if(idx < 4000){
       a[idx] = (float) threadIdx.x; // One thread operating on one index. There are total of (4127)//128 = 32 blocks, each having 128 threads. Total 4096 threads.
    }
  }
  """)
func = mod.get_function("cuda_func")
func(cuda.InOut(temp), grid=((4000+127)//128, 1), block=(128, 1, 1))
temp[-33:]

array([127.,   0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,
        10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,
        21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.],
      dtype=float32)

In [ ]:
#
# 1D grid and 2D block
#

temp = numpy.zeros((4000,))
temp2 = numpy.zeros((4000,))
temp = temp.astype(numpy.float32)
temp2 = temp2.astype(numpy.float32)
mod = SourceModule("""
  __global__ void cuda_func(float *a, float *b)
  {
    int idx = blockDim.x*blockDim.y*blockIdx.x + blockDim.x* threadIdx.y + threadIdx.x;  // Go to block + Go to row to current thread + Go to thread
    if(idx < 4000){
       a[idx] = (float) threadIdx.x; // One thread operating on one index. There are total of (4063)//64 = 63 blocks, each having (8x8)=64 threads. Total 4032 threads.
       b[idx] = (float) threadIdx.y;
    }
  }
  """)
func = mod.get_function("cuda_func")
func(cuda.InOut(temp), cuda.InOut(temp2), grid=((4000+63)//64, 1), block=(8, 8, 1))
temp[-10:], temp2[-10:]

(array([6., 7., 0., 1., 2., 3., 4., 5., 6., 7.], dtype=float32),
 array([2., 2., 3., 3., 3., 3., 3., 3., 3., 3.], dtype=float32))

In [ ]:
#
# 1D grid and 3D block
#

temp = numpy.zeros((4000,))
temp = temp.astype(numpy.float32)
temp2 = numpy.zeros((4000,))
temp2 = temp2.astype(numpy.float32)
temp3 = numpy.zeros((4000,))
temp3 = temp3.astype(numpy.float32)
mod = SourceModule("""
  __global__ void cuda_func(float *a, float *b, float *c)
  {
    int idx = blockDim.x*blockDim.y*blockDim.z*blockIdx.x +  // Go to block 
              blockDim.x*blockDim.y*threadIdx.z +            // Go to z sclice containing thread
              blockDim.x*threadIdx.y +                       // In that slice go to row containing thread
              threadIdx.x;                                   // Go to thread
    if(idx < 4000){
       a[idx] = (float) threadIdx.x; // One thread operating on one index. There are total of (4063)//64 = 63 blocks, each having (4x4x4)=64 threads. Total 4032 threads.
       b[idx] = (float) threadIdx.y;
       c[idx] = (float) threadIdx.z;
    }
  }
  """)
func = mod.get_function("cuda_func")
func(cuda.InOut(temp), cuda.InOut(temp2), cuda.InOut(temp3), grid=((4000+63)//64, 1), block=(4, 4, 4))
temp[-17:], temp2[-17:], temp3[-17:] 

(array([3., 0., 1., 2., 3., 0., 1., 2., 3., 0., 1., 2., 3., 0., 1., 2., 3.],
       dtype=float32),
 array([3., 0., 0., 0., 0., 1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3.],
       dtype=float32),
 array([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=float32))

In [ ]:
#
# 2D grid and 3D block
#

temp = numpy.zeros((4000,))
temp = temp.astype(numpy.float32)
temp2 = numpy.zeros((4000,))
temp2 = temp2.astype(numpy.float32)
temp3 = numpy.zeros((4000,))
temp3 = temp3.astype(numpy.float32)
temp4 = numpy.zeros((4000,))
temp4 = temp4.astype(numpy.float32)
temp5 = numpy.zeros((4000,))
temp5 = temp5.astype(numpy.float32)
mod = SourceModule("""
  __global__ void cuda_func(float *a, float *b, float *c, float *d, float *e)
  {
    int idx = gridDim.x*blockDim.x*blockDim.y*blockDim.z*blockIdx.y +      // Go to grid row containing block
              blockDim.x*blockDim.y*blockDim.z*blockIdx.x +                // Go to block
              blockDim.x*blockDim.y*threadIdx.z +                          // Go to slice of block containing thread
              blockDim.x*threadIdx.y +                                     // In that slice go to row containing thread
              threadIdx.x;                                                 // Go to thread
    if(idx < 4000){
       a[idx] = (float) threadIdx.x; // One thread operating on one index. There are total of (8*8) = 64 blocks, each having (4x4x4)=64 threads. Total 4096 threads.
       b[idx] = (float) threadIdx.y;
       c[idx] = (float) threadIdx.z;
       d[idx] = (float) blockIdx.x;
       e[idx] = (float) blockIdx.y;
    }
  }
  """)
func = mod.get_function("cuda_func")
func(cuda.InOut(temp), cuda.InOut(temp2), cuda.InOut(temp3), cuda.InOut(temp4), cuda.InOut(temp5), grid=(8, 8), block=(4, 4, 4))
temp[-17:], temp2[-17:], temp3[-17:], temp4[-17:], temp5[-17:]

(array([3., 0., 1., 2., 3., 0., 1., 2., 3., 0., 1., 2., 3., 0., 1., 2., 3.],
       dtype=float32),
 array([3., 0., 0., 0., 0., 1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3.],
       dtype=float32),
 array([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=float32),
 array([6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.],
       dtype=float32),
 array([7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7.],
       dtype=float32))

In [ ]:
#
# 1D grid and 1D block and only 1 thread
#

temp = numpy.zeros((4000,))
temp = temp.astype(numpy.float32)
mod = SourceModule("""
  __global__ void cuda_func(float *a)
  {
    printf("%d", threadIdx.x);
    for(int i=0; i<4000; i++){
      a[i] = i;
    }
  }
  """)
func = mod.get_function("cuda_func")
func(cuda.InOut(temp), grid=(1, 1), block=(1, 1, 1))
temp[-10:]

array([3990., 3991., 3992., 3993., 3994., 3995., 3996., 3997., 3998.,
       3999.], dtype=float32)

In [ ]:
# So on a single thread you can operate on an subarray too. You will have to configure task accordingly.